# 1 Hoeffding Inequality

Run a computer simulation for flipping 1,000 virtual fair coins. Flip each coin independently 10 times.

Note: this is a first naive implementation, below is a much faster one using binomial distribution...

In [13]:
import random
import numpy as np

In [17]:
a = np.array([1, 2, 3, 1, 2, 3])
np.where(a > 2)[0][0]

2

In [73]:

def run_experiment():
    coins_flips = []

    # 0 is tail
    # 1 is head
    
    # For 1000 coins
    for _ in range(1000):
        coin_flips = []
        
        # Flip the coin 10 times
        for _ in range(10):
            coin_flips.append(random.choice([0,1]))
        
        coins_flips.append(coin_flips)

    # c1 is the first coin flipped
    c_1 = coins_flips[0]
    ν1 = sum(c_1)/10

    # crand is a coin chosen randomly from the 1,000
    c_rand = random.choice(coins_flips)
    νrand = sum(c_rand)/10

    # cmin is the coin which had the minimum frequency of heads
    coins_heads_frequency = list(map(lambda x: sum(x), coins_flips))
    min_heads_frequency = min(coins_heads_frequency)
    a = np.array(coins_heads_frequency)
    index = np.where(a == min_heads_frequency)[0][0]
    c_min = coins_flips[index]
    νmin = sum(c_min)/10
    
    return (ν1, νrand, νmin)

In [85]:
experiments = []

# Run the experiment 100,000 times in order to get a full distribution of ν1, νrand, and νmin
NUMBER_OF_RUNS = 100000

for _ in range(NUMBER_OF_RUNS):
    experiments.append(run_experiment())

0.00503


## 1.1 The average value of νmin is closest to:

In [126]:
avg_νmin = sum(list(map(lambda x: x[2], experiments)))/NUMBER_OF_RUNS
print(f'Average νmin is {avg_νmin}, therefore closest to answer B (0.01)')

Average νmin is 0.03812299999997629, therefore closest to answer B (0.01)


## 1.1-bis Hoeffding Inequality with binomial distribution

In [115]:
import scipy.special

N = 100000
v_first = np.zeros(N)
v_min = np.zeros(N)
v_rand = np.zeros(N)

#Loop starts
for i in range(N):
    n,p = 10, 0.5
    flips = np.random.binomial(n, p, 1000)

    # Indices
    first_coin = 0
    min_coin = np.argmin(flips)
    rand_coin = np.random.randint(0,1000)

    v_first[i] = flips[first_coin]/n
    v_min[i] = flips[min_coin]/n
    v_rand[i] = flips[rand_coin]/n

#Loop ends

print(f'Average νmin is {np.average(v_min)}, therefore closest to answer D (0.5)')



Average νmin is 0.03792199999999999, therefore closest to answer D (0.5)


In [123]:
n,p = 10, 0.5
flips = np.random.binomial(n, p, 10)
print(flips)

[8 8 5 6 5 5 5 3 5 7]


### 2. Which coin(s) has a distribution of ν that satisfies the (single-bin) Hoeffding Inequality?

> The main point is that once you consider the sample, the probability becomes conditional on how this sample came out, and that could violate the 2e^{-2\epsilon^2N} bound, whereas the probability before a sample was drawn always obeys the bound.

# Linear Regression